In [1]:
import os
import pandas as pd
from google.cloud import bigquery
from google.oauth2.service_account import Credentials
import geopy.distance
import numpy as np
from pathlib import Path
from tqdm import tqdm

In [2]:
BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\dev_TIR_group2\Taipei-transit-data_hub\airflow\dags\harry_GCS_BigQuery_write_cred.json"
# BIGQUERY_CREDENTIALS_FILE_PATH = r"C:\dev_TIR101\Taipei-transit-data_hub\airflow\dags\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH
BQ_CLIENT = bigquery.Client()

In [3]:
start_date = '2024-04-16'
end_date = '2024-04-30'

In [4]:
def query_bq_to_df(client: bigquery.Client,sql_query:str) -> pd.DataFrame:
    try:
        query_job = client.query(sql_query)
        return query_job.to_dataframe()  # Convert result to DataFrame
    except Exception as e:
        raise Exception(f"Failed to query bigquery table, reason: {e}")

<h2 style="color:red;">建立新的MART表，將並每10分鐘的車輛數groupby成每小時</h2>
<h2 style="color:red;">並選定日期期間</h2>

In [13]:
def MART_bike_realtime_select_date_create(
        start_date :str,
        end_date :str,
        create_dataset_name: str,
        create_table_name: str,
        source_dataset_name:str,
        source_table_name: str, 
        client: bigquery.Client):
    """create MART table for bike_realtime station information"""
    query_job = client.query(
        f"""
    CREATE OR REPLACE TABLE `{create_dataset_name}.{create_table_name}` AS
    SELECT 
        `bike_station_id`,
        EXTRACT(DATE FROM `source_time`) AS date,
        EXTRACT(HOUR FROM `source_time`) AS hour,
        AVG(`aval_bike`) AS aval_bike_mean,
        AVG(`aval_bike`) AS aval_space_mean
    FROM `{source_dataset_name}.{source_table_name}`
    WHERE `source_time` BETWEEN TIMESTAMP('{start_date}') AND TIMESTAMP('{end_date}') 
    GROUP BY bike_station_id , date , hour
    """
    )
    query_job.result()
    print(f"{create_dataset_name}.{create_table_name} has been created")

In [14]:
MART_bike_realtime_select_date_create(
   start_date ='2024-04-16',
   end_date = '2024-04-30',
   create_dataset_name="ANDY_ETL_MART",
   create_table_name="MART_youbike_bike_groupby_date_hour",
   source_dataset_name = "ANDY_ETL_FACT",
   source_table_name="FACT_bike_realtime", 
   client=BQ_CLIENT)

ANDY_ETL_MART.MART_youbike_bike_groupby_date_hour has been created


<h2 style="color:red;">建立MART層中Youbike站點所對應到的捷運站最近距離</h2>
<h4 style="color:red;">距離表只有捷運ID 需Join捷運站的中文站名</h4>

In [15]:
def MART_youbike_min_distance_create(create_dataset_name:str,
                             create_table_name:str,
                             source_dataset_name:str,
                             source_table_name:str,
                             mrt_info_dataset_name:str,
                             mrt_info_table_name:str,
                             client:bigquery.Client):
    query_job = client.query(
    f"""
    CREATE OR REPLACE TABLE `{create_dataset_name}.{create_table_name}` AS
    SELECT t1.* ,t2.mrt_station_name
    FROM
        (SELECT
            `bike_station_id`,
            `mrt_station_id`,
            `distance`
        FROM
            (SELECT 
                `bike_station_id`,
                `mrt_station_id`,
                `distance`,
                ROW_NUMBER() OVER (PARTITION BY `bike_station_id` ORDER BY `distance`) AS `row_num`
            FROM `{source_dataset_name}.{source_table_name}`) 
        WHERE row_num=1) AS t1
    LEFT JOIN
        (SELECT 
            `mrt_station_id`,
            `station_name` AS `mrt_station_name`
        FROM `{mrt_info_dataset_name}.{mrt_info_table_name}`) AS t2
    ON t1.mrt_station_id = t2.mrt_station_id
        
    ;
    """
    )
    query_job.result()
    print(f"{create_dataset_name}.{create_table_name} has been created")

In [16]:
MART_youbike_min_distance_create(create_dataset_name="ANDY_ETL_MART",
                                 create_table_name="MART_youbike_mrt_distance",
                                 source_dataset_name="ETL_DIM",
                                 source_table_name="DIM_youbike_mrt_distance",
                                mrt_info_dataset_name="MRT_GCS_to_BQ_SRC_ODS_DIM",
                                mrt_info_table_name="DIM_MRT_static_data",
                                 client=BQ_CLIENT)

ANDY_ETL_MART.MART_youbike_mrt_distance has been created


<h2 style="color:red;">建立MART層中捷運站歷史個時間點的進出站人次</h2>

In [17]:
def MART_MRT_history_groupby_create(
    start_date :str,
    end_date :str,
    create_dataset_name:str,
    create_table_name:str,
    source_dataset_name:str,
    source_table_name:str,
    client:bigquery.Client):
    query_job = client.query(
    f"""
    CREATE OR REPLACE TABLE `{create_dataset_name}.{create_table_name}` AS
    SELECT t_enter.date,t_enter.hour,t_enter.mrt_station_name,t_enter.enter_num,t_exit.exit_num
    FROM
        (SELECT
            date,
            hour,
            mrt_station_name_enter AS mrt_station_name,
            SUM(visitors_num) AS enter_num
        FROM 
            `{source_dataset_name}.{source_table_name}`
        WHERE DATE(date) >= '{start_date}' AND  DATE(date) <='{end_date}'
        GROUP BY date,hour,mrt_station_name_enter) AS t_enter
    FULL OUTER JOIN
        (SELECT
            date,
            hour,
            mrt_station_name_exit AS mrt_station_name,
            SUM(visitors_num) AS exit_num
        FROM 
            `{source_dataset_name}.{source_table_name}`
        WHERE DATE(date) >= '{start_date}' AND  DATE(date) <='{end_date}'
        GROUP BY date,hour,mrt_station_name_exit) AS t_exit
    ON t_enter.date = t_exit.date AND t_enter.hour = t_exit.hour AND t_enter.mrt_station_name = t_exit.mrt_station_name
    ORDER BY t_enter.date,t_enter.hour,t_enter.mrt_station_name DESC;
    """)
    query_job.result()
    print(f"{create_dataset_name}.{create_table_name} has been created")

In [18]:
MART_MRT_history_groupby_create(
    start_date="2024-04-01",
    end_date="2024-04-30",
    create_dataset_name="ANDY_ETL_MART",
    create_table_name="MART_mrt_history_2024_04_grby",
    source_dataset_name="MRT_history",
    source_table_name="ODS_MRT_history_usage",
    client=BQ_CLIENT)

ANDY_ETL_MART.MART_mrt_history_2024_04_grby has been created


<h2 style="color:red;">建立MART層中youbike站點的完整資訊</h2>
<h4 style="color:red;">join站點資訊以及time table(日期星期對應) </h4>

In [19]:
def MART_youbike_bike_realtime_and_info_create(
      create_dataset_name: str,
      create_table_name: str,
      source_dataset_name_left:str,
      source_table_name_left: str,
      source_dataset_name_right:str,
      source_table_name_right: str,
      time_table_dataset:str,
      time_table_table:str,
      client: bigquery.Client):
    """create MART table for bike_realtime with station information"""
    query_job = client.query(
        f"""
    CREATE OR REPLACE TABLE `{create_dataset_name}.{create_table_name}` AS
    SELECT 
      t1.* , 
      t2.station_name,
      t2.total_space,
      t2.lat,
      t2.lng,
      t3.day_of_week,
      t3.day_of_week_name
    FROM
      (SELECT 
        *
      FROM `{source_dataset_name_left}.{source_table_name_left}`) AS t1
    LEFT JOIN
      (SELECT
         *
      FROM `{source_dataset_name_right}.{source_table_name_right}`) AS t2
    ON t1.bike_station_id = t2.bike_station_id 
    LEFT JOIN
      (SELECT
         *
      FROM `{time_table_dataset}.{time_table_table}`) AS t3
    ON t1.date = t3.date 
      ;
    """
    )
    query_job.result()
    print(f"{create_dataset_name}.{create_table_name} has been created")

In [20]:
MART_youbike_bike_realtime_and_info_create(
    create_dataset_name="ANDY_ETL_MART",
    create_table_name="MART_youbike_bike_realtime_and_info",
    source_dataset_name_left="ANDY_ETL_MART",
    source_table_name_left="MART_youbike_bike_groupby_date_hour",
    source_dataset_name_right="ANDY_ETL_DIM",
    source_table_name_right="DIM_bike_station",
    time_table_dataset="ANDY_ETL_DIM",
    time_table_table="DIM_time_table",
    client=BQ_CLIENT)

ANDY_ETL_MART.MART_youbike_bike_realtime_and_info has been created


<h2 style="color:red;">建立MART層中的pipeline完成</h2>
<h4 style="color:red;">將youbike站點的資訊 join最近距離捷運站表  在join該捷運歷史該時段的人流</h4>

In [43]:
def MART_bike_pipeline_create(
        client: bigquery.Client,
        create_dataset_name: str,
        create_table_name: str,
        MART_dataset_name:str,
        MART_youbike_bike_realtime_and_info="MART_youbike_bike_realtime_and_info",
        MART_youbike_mrt_distance="MART_youbike_mrt_distance",
        MART_mrt_history_2024_04_grby ="MART_mrt_history_2024_04_grby"):
    """create MART table for bike_realtime with station information"""
    query_job = client.query(
        f"""
    CREATE OR REPLACE TABLE `{create_dataset_name}.{create_table_name}` AS
    SELECT 
      t1.* , 
      t2.mrt_station_id,
      t2.mrt_station_name,
      t2.distance,
      t3.enter_num,
      t3.exit_num,
      t1.aval_bike_mean/t1.total_space AS `bike_rate`,
      t3.exit_num + t3.enter_num AS `total_visitors`,
      t3.exit_num - t3.enter_num AS `ex_enter_diff`
    FROM
      (SELECT 
        *
      FROM `{MART_dataset_name}.{MART_youbike_bike_realtime_and_info}`) AS t1
    LEFT JOIN
      (SELECT
         *
      FROM `{MART_dataset_name}.{MART_youbike_mrt_distance}`) AS t2
    ON t1.bike_station_id = t2.bike_station_id 
    LEFT JOIN
      (SELECT
         *
      FROM `{MART_dataset_name}.{MART_mrt_history_2024_04_grby}`) AS t3
    ON t2.mrt_station_name = t3.mrt_station_name
    AND t1.date = t3.date
    AND t1.hour = t3.hour
      ;
    """
    )
    query_job.result()
    print(f"{create_dataset_name}.{create_table_name} has been created")

In [44]:
MART_bike_pipeline_create(
        client=BQ_CLIENT,
        create_dataset_name="ANDY_ETL_MART",
        create_table_name="MART_youbike_correlation_pipeline",
        MART_dataset_name="ANDY_ETL_MART",
        MART_youbike_bike_realtime_and_info="MART_youbike_bike_realtime_and_info",
        MART_youbike_mrt_distance="MART_youbike_mrt_distance",
        MART_mrt_history_2024_04_grby ="MART_mrt_history_2024_04_grby")

ANDY_ETL_MART.MART_youbike_correlation_pipeline has been created


In [45]:
sql_query = """SELECT * FROM `ANDY_ETL_MART.MART_youbike_correlation_pipeline`"""

df = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query)
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,station_name,total_space,lat,lng,day_of_week,day_of_week_name,mrt_station_id,mrt_station_name,distance,enter_num,exit_num,bike_rate,total_visitors,ex_enter_diff
0,500105029,2024-04-21,18,14.333333,14.333333,YouBike2.0_一壽橋,28,24.97837,121.55548,1,Sunday,G03,七張,1.300606,729,1072,0.511905,1801,343
1,500105029,2024-04-21,9,11.000000,11.000000,YouBike2.0_一壽橋,28,24.97837,121.55548,1,Sunday,G03,七張,1.300606,651,365,0.392857,1016,-286
2,500105029,2024-04-28,0,17.000000,17.000000,YouBike2.0_一壽橋,28,24.97837,121.55548,1,Sunday,G03,七張,1.300606,41,123,0.607143,164,82
3,500105029,2024-04-28,8,21.166667,21.166667,YouBike2.0_一壽橋,28,24.97837,121.55548,1,Sunday,G03,七張,1.300606,637,253,0.755952,890,-384
4,500105029,2024-04-28,12,5.333333,5.333333,YouBike2.0_一壽橋,28,24.97837,121.55548,1,Sunday,G03,七張,1.300606,852,584,0.190476,1436,-268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
438546,500106110,2024-04-20,21,0.833333,0.833333,YouBike2.0_捷運善導寺站(3號出口)(忠孝東路側),31,25.04467,121.52357,7,Saturday,BL13,善導寺,0.031807,915,518,0.026882,1433,-397
438547,500106110,2024-04-27,6,9.666667,9.666667,YouBike2.0_捷運善導寺站(3號出口)(忠孝東路側),31,25.04467,121.52357,7,Saturday,BL13,善導寺,0.031807,78,257,0.311828,335,179
438548,500106110,2024-04-20,15,15.166667,15.166667,YouBike2.0_捷運善導寺站(3號出口)(忠孝東路側),31,25.04467,121.52357,7,Saturday,BL13,善導寺,0.031807,1440,1117,0.489247,2557,-323
438549,500106110,2024-04-27,4,5.666667,5.666667,YouBike2.0_捷運善導寺站(3號出口)(忠孝東路側),31,25.04467,121.52357,7,Saturday,BL13,善導寺,0.031807,<NA>,<NA>,0.182796,<NA>,<NA>


In [46]:
import math
def logit_transform(x):
    if x==0:
        x = 10**(-3)
    elif x>=1:
        x=0.9999
    return(math.log(x/(1-x)))
def square_root_transform(x):
    if x>=1:
        x=1
    return (x**2)
def log_transform(x):
    if x>=1:
        x=1
    elif x==0:
        x = 10**(-3)
    return(math.log(x))
def inverse_transform(x):
    if x>=1:
        x=1
    elif x==0:
        x = 10**(-3)
    return(1/x)

In [47]:
df["bike_rate"] = df["bike_rate"].apply(logit_transform)
df = df.loc[~df["hour"].isin([22,23,0,1,2,3,4,5]),]
bk_ids = list(df["bike_station_id"].unique())
corr_df = pd.DataFrame(columns=["bike_station_id",
                                "corr_enter_bike",
                                "corr_exit_bike",
                                "corr_total_visitors",
                                "corr_ex_enter_diff",
                                "corr_enter_exit"])

In [48]:
for bk_id in bk_ids:
    tem = df.loc[df["bike_station_id"]==bk_id,]
    cor_matrix = tem.loc[:,["bike_rate","enter_num","total_visitors","ex_enter_diff","exit_num"]].corr()
    corr_df_tem = pd.DataFrame({
        "bike_station_id":[bk_id],
        "corr_enter_bike":[cor_matrix.loc["enter_num","bike_rate"]],
        "corr_exit_bike":cor_matrix.loc["exit_num","bike_rate"],
        "corr_total_visitors":cor_matrix.loc["total_visitors","bike_rate"],
        "corr_ex_enter_diff":cor_matrix.loc["ex_enter_diff","bike_rate"],
        "corr_enter_exit":cor_matrix.loc["exit_num","enter_num"]
    })
    corr_df = pd.concat([corr_df,corr_df_tem],axis=0)

C:\Users\andy\AppData\Local\Temp\ipykernel_17088\3527425555.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  corr_df = pd.concat([corr_df,corr_df_tem],axis=0)


In [49]:
corr_df.sort_values(["corr_ex_enter_diff"],ascending=True,inplace=True)
corr_df

,bike_station_id,corr_enter_bike,corr_exit_bike,corr_total_visitors,corr_ex_enter_diff,corr_enter_exit
0,500119048,0.569666,-0.056100,0.347806,-0.738163,0.578933
0,500113040,0.090182,-0.503709,-0.264032,-0.681918,0.583025
0,500105005,0.332880,-0.609534,-0.150617,-0.636143,-0.046347
0,500101181,0.434360,-0.120876,0.228895,-0.629833,0.578933
0,500110026,0.513630,-0.167003,0.256376,-0.579524,0.256228
...,...,...,...,...,...,...
0,500101127,NaN,NaN,NaN,NaN,0.354560
0,500105062,NaN,NaN,NaN,NaN,0.069693
0,500103024,NaN,NaN,NaN,NaN,NaN
0,500101159,NaN,NaN,NaN,NaN,0.354560


In [ ]:
###下面為測試用

In [24]:
sql_query = """SELECT * FROM `ANDY_ETL_MART.MART_youbike_bike_groupby_date_hour`"""

df = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query)
df

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean
0,500108169,2024-04-16,22,0.0,0.0
1,500119067,2024-04-16,22,0.0,0.0
2,500112010,2024-04-16,22,0.0,0.0
3,500113068,2024-04-16,22,0.0,0.0
4,500112091,2024-04-16,22,0.0,0.0
...,...,...,...,...,...
438546,500113035,2024-04-29,20,43.0,43.0
438547,500119048,2024-04-29,20,45.0,45.0
438548,500104031,2024-04-29,20,47.0,47.0
438549,500110014,2024-04-29,20,49.0,49.0


In [25]:
sql_query = """SELECT * FROM `ANDY_ETL_MART.MART_youbike_bike_realtime_and_info`"""

df = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query)
df.head()

,bike_station_id,date,hour,aval_bike_mean,aval_space_mean,station_name,total_space,lat,lng,day_of_week,day_of_week_name
0,500105029,2024-04-16,22,26.833333,26.833333,YouBike2.0_一壽橋,28,24.97837,121.55548,3,Tuesday
1,500105029,2024-04-16,23,25.333333,25.333333,YouBike2.0_一壽橋,28,24.97837,121.55548,3,Tuesday
2,500112022,2024-04-16,22,14.666667,14.666667,YouBike2.0_三張犁,56,25.03452,121.55760,3,Tuesday
3,500112022,2024-04-16,23,13.333333,13.333333,YouBike2.0_三張犁,56,25.03452,121.55760,3,Tuesday
4,500106059,2024-04-16,22,0.833333,0.833333,YouBike2.0_中山堂,43,25.04412,121.51025,3,Tuesday


In [18]:
sql_query = """SELECT * FROM `ANDY_ETL_MART.MART_youbike_mrt_distance`"""

df = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query)
df.head()

,bike_station_id,mrt_station_id,distance,mrt_station_name
0,500105030,G03,1.459308,七張
1,500105034,G03,1.537141,七張
2,500105029,G03,1.300606,七張
3,500105088,G03,1.201386,七張
4,500103053,R11,0.106156,中山


In [24]:
sql_query = """SELECT * FROM `ANDY_ETL_MART.MART_mrt_history_2024_04_grby`"""

df = query_bq_to_df(client=BQ_CLIENT,sql_query=sql_query)
df.head()

,date,hour,mrt_station,enter_num,exit_num
0,2024-04-01,0,龍山寺,144,153
1,2024-04-01,0,麟光,15,21
2,2024-04-01,0,頭前庄,10,35
3,2024-04-01,0,頂溪,85,219
4,2024-04-01,0,頂埔,13,100
